In [1]:
from bs4 import BeautifulSoup
import re, time

import urllib.request


'''
<a class="carsListItemNameLink" href="https://www.autotalli.com/vaihtoauto/35897954/Renault/Laguna/2000?pos=17&amp;page=5&amp;searchType=usedCar">Renault Laguna BREAK CONFORT 1.6</a>
'''

class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"

def getDataFromLink(link):
    opener = AppURLopener()
    response = opener.open(link).read()
    soup = BeautifulSoup(response, "lxml")
    dataHtml = soup.findAll("div", {"class": "listContent used"})

    linkdataRaw = [a.find_all("a", {"class": "carsListItemNameLink"}) for a in dataHtml]
    links = [a['href'] for a in linkdataRaw[0]]
    
    #print(dataHtml)
    
    namedataRaw = [a.find_all("a", {"class": "carsListItemNameLink"}) for a in dataHtml]
    names=[]
    for n in namedataRaw[0]:
        names.append(n.string)
    
    pricedataRaw = [a.find_all("div", {"class": "carsListItemCarDetailBottomContainer"}) for a in dataHtml]
    pricekmyeardata = [a.find_all("div", {"class": "carsListItemCarBottomContainerItem"}) for a in pricedataRaw[0]]
    prices=[]
    years=[]
    kms=[]
    for p in pricekmyeardata:
        prices.append(p[0].string)
        years.append(p[2].string)
        kms.append(p[1].string)

    
    complete_data=[]
    for i in range(0,len(links)):
        complete_data.append((links[i],
        names[i],
        prices[i],
        kms[i],
        years[i]))
    return(complete_data)

import csv
def writeToCsv(file,data):
    with open(file,'a') as f:
        wr = csv.writer(f, dialect='excel')
        wr.writerows(data)
    
link='https://www.autotalli.com/vaihtoautot/listaa/sivu/5'
data=getDataFromLink(link)
#writeToCsv("carPrices.csv", data)
#print(data)

C:\Users\mpalonie\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  from ipykernel import kernelapp as app


In [4]:
import datetime
import psutil
import os

time=str(datetime.datetime.now().date())
print(time)
filename=time + " autotalli.csv"
f=open(filename,'a')
f.close()

raw_link='https://www.autotalli.com/vaihtoautot/listaa/sivu/'
last_page=2400

failed=0
for i in range(1,last_page):
    try:
        print("Progress "+str(i/last_page))
        link=raw_link+str(i)
        data=getDataFromLink(link)
        writeToCsv(filename, data)
    except KeyboardInterrupt:
        break
    except:
        print("Round failed ",i)
        failed=failed+1
print("Finished")
print("Failed ", failed)
print("Succeeded ", last_page-failed)

2018-10-23
Progress 0.0004166666666666667


C:\Users\mpalonie\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  from ipykernel import kernelapp as app


Progress 0.0008333333333333334
Progress 0.00125
Progress 0.0016666666666666668
Progress 0.0020833333333333333
Progress 0.0025
Progress 0.002916666666666667
Progress 0.0033333333333333335
Progress 0.00375
Progress 0.004166666666666667
Progress 0.004583333333333333
Progress 0.005
Progress 0.005416666666666667
Progress 0.005833333333333334
Progress 0.00625
Progress 0.006666666666666667
Progress 0.007083333333333333
Progress 0.0075
Progress 0.007916666666666667
Progress 0.008333333333333333
Progress 0.00875
Progress 0.009166666666666667
Progress 0.009583333333333333
Progress 0.01
Progress 0.010416666666666666
Progress 0.010833333333333334
Progress 0.01125
Progress 0.011666666666666667
Progress 0.012083333333333333
Progress 0.0125
Progress 0.012916666666666667
Progress 0.013333333333333334
Progress 0.01375
Progress 0.014166666666666666
Progress 0.014583333333333334
Progress 0.015
Progress 0.015416666666666667
Progress 0.015833333333333335
Progress 0.01625
Progress 0.016666666666666666
Progr

Progress 0.13791666666666666
Progress 0.13833333333333334
Progress 0.13875
Progress 0.13916666666666666
Progress 0.13958333333333334
Progress 0.14
Progress 0.14041666666666666
Progress 0.14083333333333334
Progress 0.14125
Progress 0.14166666666666666
Progress 0.14208333333333334
Progress 0.1425
Progress 0.14291666666666666
Progress 0.14333333333333334
Progress 0.14375
Progress 0.14416666666666667
Progress 0.14458333333333334
Progress 0.145
Progress 0.14541666666666667
Progress 0.14583333333333334
Progress 0.14625
Progress 0.14666666666666667
Progress 0.14708333333333334
Progress 0.1475
Progress 0.14791666666666667
Progress 0.14833333333333334
Progress 0.14875
Progress 0.14916666666666667
Progress 0.14958333333333335
Progress 0.15
Progress 0.15041666666666667
Progress 0.15083333333333335
Progress 0.15125
Progress 0.15166666666666667
Progress 0.15208333333333332
Progress 0.1525
Progress 0.15291666666666667
Progress 0.15333333333333332
Progress 0.15375
Progress 0.15416666666666667
Progres

Progress 0.27625
Progress 0.27666666666666667
Progress 0.27708333333333335
Progress 0.2775
Progress 0.27791666666666665
Progress 0.2783333333333333
Progress 0.27875
Progress 0.2791666666666667
Progress 0.27958333333333335
Progress 0.28
Progress 0.28041666666666665
Progress 0.2808333333333333
Progress 0.28125
Progress 0.2816666666666667
Progress 0.28208333333333335
Progress 0.2825
Progress 0.28291666666666665
Progress 0.2833333333333333
Progress 0.28375
Progress 0.2841666666666667
Progress 0.28458333333333335
Progress 0.285
Progress 0.28541666666666665
Progress 0.28583333333333333
Progress 0.28625
Progress 0.2866666666666667
Progress 0.28708333333333336
Progress 0.2875
Progress 0.28791666666666665
Progress 0.28833333333333333
Progress 0.28875
Progress 0.2891666666666667
Progress 0.28958333333333336
Progress 0.29
Progress 0.29041666666666666
Progress 0.29083333333333333
Progress 0.29125
Progress 0.2916666666666667
Progress 0.2920833333333333
Progress 0.2925
Progress 0.29291666666666666
P

...

Round failed  1455
Progress 0.6066666666666667
Round failed  1456
Progress 0.6070833333333333
Round failed  1457
Progress 0.6075
Round failed  1458
Progress 0.6079166666666667
Round failed  1459
Progress 0.6083333333333333
Round failed  1460
Progress 0.60875
Round failed  1461
Progress 0.6091666666666666
Round failed  1462
Progress 0.6095833333333334
Round failed  1463
Progress 0.61
Round failed  1464
Progress 0.6104166666666667
Round failed  1465
Progress 0.6108333333333333
Round failed  1466
Progress 0.61125
Round failed  1467
Progress 0.6116666666666667
Round failed  1468
Progress 0.6120833333333333
Round failed  1469
Progress 0.6125
Round failed  1470
Progress 0.6129166666666667
Round failed  1471
Progress 0.6133333333333333
Round failed  1472
Progress 0.61375
Round failed  1473
Progress 0.6141666666666666
Round failed  1474
Progress 0.6145833333333334
Round failed  1475
Progress 0.615
Round failed  1476
Progress 0.6154166666666666
Round failed  1477
Progress 0.6158333333333333
Rou

Progress 0.6879166666666666
Round failed  1651
Progress 0.6883333333333334
Round failed  1652
Progress 0.68875
Round failed  1653
Progress 0.6891666666666667
Round failed  1654
Progress 0.6895833333333333
Round failed  1655
Progress 0.69
Round failed  1656
Progress 0.6904166666666667
Round failed  1657
Progress 0.6908333333333333
Round failed  1658
Progress 0.69125
Round failed  1659
Progress 0.6916666666666667
Round failed  1660
Progress 0.6920833333333334
Round failed  1661
Progress 0.6925
Round failed  1662
Progress 0.6929166666666666
Round failed  1663
Progress 0.6933333333333334
Round failed  1664
Progress 0.69375
Round failed  1665
Progress 0.6941666666666667
Round failed  1666
Progress 0.6945833333333333
Round failed  1667
Progress 0.695
Round failed  1668
Progress 0.6954166666666667
Round failed  1669
Progress 0.6958333333333333
Round failed  1670
Progress 0.69625
Round failed  1671
Progress 0.6966666666666667
Round failed  1672
Progress 0.6970833333333334
Round failed  1673
Pr

Round failed  1841
Progress 0.7675
Round failed  1842
Progress 0.7679166666666667
Round failed  1843
Progress 0.7683333333333333
Round failed  1844
Progress 0.76875
Round failed  1845
Progress 0.7691666666666667
Round failed  1846
Progress 0.7695833333333333
Round failed  1847
Progress 0.77
Round failed  1848
Progress 0.7704166666666666
Round failed  1849
Progress 0.7708333333333334
Round failed  1850
Progress 0.77125
Round failed  1851
Progress 0.7716666666666666
Round failed  1852
Progress 0.7720833333333333
Round failed  1853
Progress 0.7725
Round failed  1854
Progress 0.7729166666666667
Round failed  1855
Progress 0.7733333333333333
Round failed  1856
Progress 0.77375
Round failed  1857
Progress 0.7741666666666667
Round failed  1858
Progress 0.7745833333333333
Round failed  1859
Progress 0.775
Round failed  1860
Progress 0.7754166666666666
Round failed  1861
Progress 0.7758333333333334
Round failed  1862
Progress 0.77625
Round failed  1863
Progress 0.7766666666666666
Round failed  

Round failed  2042
Progress 0.85125
Round failed  2043
Progress 0.8516666666666667
Round failed  2044
Progress 0.8520833333333333
Round failed  2045
Progress 0.8525
Round failed  2046
Progress 0.8529166666666667
Round failed  2047
Progress 0.8533333333333334
Round failed  2048
Progress 0.85375
Round failed  2049
Progress 0.8541666666666666
Round failed  2050
Progress 0.8545833333333334
Round failed  2051
Progress 0.855
Round failed  2052
Progress 0.8554166666666667
Round failed  2053
Progress 0.8558333333333333
Round failed  2054
Progress 0.85625
Round failed  2055
Progress 0.8566666666666667
Round failed  2056
Progress 0.8570833333333333
Round failed  2057
Progress 0.8575
Round failed  2058
Progress 0.8579166666666667
Round failed  2059
Progress 0.8583333333333333
Round failed  2060
Progress 0.85875
Round failed  2061
Progress 0.8591666666666666
Round failed  2062
Progress 0.8595833333333334
Round failed  2063
Progress 0.86
Round failed  2064
Progress 0.8604166666666667
Round failed  

Round failed  2243
Progress 0.935
Round failed  2244
Progress 0.9354166666666667
Round failed  2245
Progress 0.9358333333333333
Round failed  2246
Progress 0.93625
Round failed  2247
Progress 0.9366666666666666
Round failed  2248
Progress 0.9370833333333334
Round failed  2249
Progress 0.9375
Round failed  2250
Progress 0.9379166666666666
Round failed  2251
Progress 0.9383333333333334
Round failed  2252
Progress 0.93875
Round failed  2253
Progress 0.9391666666666667
Round failed  2254
Progress 0.9395833333333333
Round failed  2255
Progress 0.94
Round failed  2256
Progress 0.9404166666666667
Round failed  2257
Progress 0.9408333333333333
Round failed  2258
Progress 0.94125
Round failed  2259
Progress 0.9416666666666667
Round failed  2260
Progress 0.9420833333333334
Round failed  2261
Progress 0.9425
Round failed  2262
Progress 0.9429166666666666
Round failed  2263
Progress 0.9433333333333334
Round failed  2264
Progress 0.94375
Round failed  2265
Progress 0.9441666666666667
Round failed  